In [2]:
import os
os.environ["MLFLOW_TRACHING_URI"] = "https://dagshub.com/MahammadRafi06/MLops_Ds1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "MahammadRafi06"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "5997c2edac04d59b6f2da7e2a0b700721ebe7cf4"

In [3]:
%pwd

'/home/mrafi/Desktop/Books/Bootcamp/E2EMLOps/18-e2e/MLops_Ds1/research'

In [4]:
os.chdir("../")
%pwd

'/home/mrafi/Desktop/Books/Bootcamp/E2EMLOps/18-e2e/MLops_Ds1'

In [11]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:
    root_dir:  Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    mlflow_uri: str
    target_column: str
    

In [ ]:
from src.data_science.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH
from src.data_science.utils.common import read_yaml, create_dir, save_json
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, schema_file_path= SCHEMA_FILE_PATH, params_file_path=PARAMS_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_dir([self.config.artifacts_root])
    
    def get_modelevaludation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet
        create_dir([config.root_dir])
        modelevaluation_config = ModelEvaluationConfig(
                root_dir= config.root_dir,
                test_data_path = config.test_data_path,
                model_path = config.model_path,
                metric_file_name = config.metric_file_name,
                all_params = params,
                mlflow_uri = os.getenv("MLFLOW_TRACHING_URI"),
                target_column = schema
        )
        return modelevaluation_config
        

In [23]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [32]:

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evalauation_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    
    def model_evaluation(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop(columns=[self.config.target_column.name], axis=1)
        test_y = test_data[self.config.target_column.name]
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.evalauation_metric(test_y, predicted_qualities)
            
            scores = {"rmse":rmse, "mae":mae, "r2":r2}
            
            save_json(path=Path(self.config.metric_file_name), data=scores)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            
            tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model1", registered_model_name="Best ElaticNet", input_example=test_x)
            else:
                mlflow.sklearn.log_model(model, "model1")
        


In [33]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_modelevaludation_config()
    data_transofrmation = ModelEvaluation(model_evaluation_config)
    data_transofrmation.model_evaluation()
except Exception as e:
    raise e 

2025-05-03 21:12:25,638, common.py, 19, INFO, config/config.yaml file loaded sucessfully
INFO:data_science:config/config.yaml file loaded sucessfully
2025-05-03 21:12:25,640, common.py, 19, INFO, params.yaml file loaded sucessfully
INFO:data_science:params.yaml file loaded sucessfully
2025-05-03 21:12:25,643, common.py, 19, INFO, schema.yaml file loaded sucessfully
INFO:data_science:schema.yaml file loaded sucessfully
2025-05-03 21:12:25,644, common.py, 32, INFO, Directory artifacts created
INFO:data_science:Directory artifacts created
2025-05-03 21:12:25,645, common.py, 32, INFO, Directory artifacts/model_evaluation created
INFO:data_science:Directory artifacts/model_evaluation created
2025-05-03 21:12:25,817, common.py, 38, INFO, data saved to artifacts/model_evaluation/metric.json
INFO:data_science:data saved to artifacts/model_evaluation/metric.json
Registered model 'Best ElaticNet' already exists. Creating a new version of this model...
2025/05/03 21:12:36 INFO mlflow.store.model_

🏃 View run likeable-ant-731 at: https://dagshub.com/MahammadRafi06/MLops_Ds1.mlflow/#/experiments/0/runs/94cad6173eff455f8a11b8769529f023
🧪 View experiment at: https://dagshub.com/MahammadRafi06/MLops_Ds1.mlflow/#/experiments/0


In [5]:
from src.data_science.utils.common import load_envfile
from pathlib import Path
import os
path= Path(os.path.join('/home/mrafi/Desktop/Books/Bootcamp/E2EMLOps/18-e2e/MLops_Ds1', ".env"))
envs = dict(load_envfile(path))

In [6]:
envs

{'"MLFLOW_TRACHING_URI"': 'https://dagshub.com/MahammadRafi06/MLops_Ds1.mlflow',
 '"MLFLOW_TRACKING_PASSWORD"': '5997c2edac04d59b6f2da7e2a0b700721ebe7cf4',
 '"MLFLOW_TRACKING_USERNAME"': 'MahammadRafi06'}